В данном блокноте объединены данные из трех источников: категоризация ChatGPT, VirusTotal и AlphaMountain. Это позволяет создать единый источник для определения наиболее релевантной категории каждого сайта. В процессе работы реализованы следующие шаги:



1. Загрузка и обработка данных
Загрузка таблиц из каждого источника, удаление дубликатов и унификация текстовых данных (перевод в нижний регистр).
Объединение данных всех источников на основе поля url.
2. Определение итоговой категории
Для каждого сайта определена наиболее популярная категория на основе совпадений данных из всех источников. Если категория отсутствует, используется значение из категоризации ChatGPT.
3. Матрица совпадений категорий
Построена матрица совпадений категорий между источниками, которая отображает:
Частоту совпадений между категориями для каждой пары источников.
Каждая ячейка показывает, сколько раз категории двух источников совпадали для всех обработанных URL.
По строкам и столбцам представлены все категории (например, cats gpt, BitDefender, Dr.Web, и др.).
Матрица позволяет выявить наиболее согласованные источники, что важно для понимания надежности и согласованности данных.
Пример интерпретации: значение 1337 в ячейке между cats base и cats gpt указывает на то, что данные этих источников совпали для 1337 сайтов.
Матрица сохранена в файл category_match_matrix.xlsx.
4. Регулярные выражения для анализа сайтов
Создана функция для генерации регулярных выражений на основе req_host, что позволяет работать с IP-адресами и доменными именами.
Преобразованная таблица содержит поля для удобного анализа сайтов: req_host_combined, url_regex_mod, url и итоговая категория cat_popular.

1. Загрузка библиотек

In [1]:
import pandas as pd
import re
from collections import Counter
import numpy as np


2. Выгрузка данных

In [2]:
results_month_api = pd.read_csv('новая категоризация аpi virustotal.csv')

In [3]:
results_month_gpt = pd.read_csv('category_result_gpt.csv')

In [4]:
# Определяем базовое имя файла и список для хранения DataFrame
base_filename = 'категоризация alpha_'  # Базовое имя файлов
dataframes = []  # Список для сохранения DataFrame

# Загружаем каждую из пяти таблиц и добавляем их в список
for i in range(5):  # Перебираем числа от 0 до 4
    filename = f'{base_filename}{i}.csv'  # Формируем имя файла
    df = pd.read_csv(filename)  # Читаем файл в DataFrame
    dataframes.append(df)  # Добавляем DataFrame в список

# Объединяем все DataFrame в один
results_month_alpha = pd.concat(dataframes, ignore_index=True)  # Конкатенация с игнорированием индексов

# Сохраняем объединенный DataFrame
results_month_alpha.to_csv('results_month_alpha.csv', index=False)  # Сохраняем в CSV без индексов



3. Предобрботка данных

In [5]:
results_month_alpha = results_month_alpha.drop_duplicates()

In [6]:
results_month_gpt['cats base'] = results_month_gpt['cats base'].str.lower()
results_month_gpt['cats gpt'] = results_month_gpt['cats gpt'].str.lower()
results_month_alpha['alpha_1'] = results_month_alpha['alpha_1'].str.lower()
results_month_alpha['alpha_2'] = results_month_alpha['alpha_2'].str.lower()


In [7]:
results_month_api['url'].drop_duplicates()


0              www.instagram.com
1                 www.google.com
2                    www.ozon.ru
3                  149.154.171.5
4                www.bot365.tips
                  ...           
2852            www.lordfilm.art
2853         www.vega-factory.ru
2854              23.227.151.122
2855           www.datadoghq.com
2856    www.googleadservices.com
Name: url, Length: 2857, dtype: object

In [8]:
results_month_gpt['url'].drop_duplicates()

0       www.googleadservices.com
1              www.instagram.com
2                 www.google.com
3                    www.ozon.ru
4                  149.154.171.5
                  ...           
4425              173.194.221.84
4426            www.lordfilm.art
4427         www.vega-factory.ru
4428              23.227.151.122
4430           www.datadoghq.com
Name: url, Length: 2902, dtype: object

In [9]:
df_temp = results_month_gpt.merge(results_month_api, how = 'left',  on = 'url')

In [10]:
df = df_temp.merge(results_month_alpha, how = 'left',  on = 'url')

4. Определение итоговой категории

Этот код определяет функцию get_resultant_category, которая вычисляет итоговую категорию для каждой строки DataFrame на основе данных из нескольких источников. Извлекаются значения из указанных столбцов, исключаются пустые значения и категории 'uncategorized', после чего определяется наиболее часто встречающаяся категория с помощью Counter. Если подходящих категорий нет, используется значение из столбца 'cats gpt'. Итоговая категория сохраняется в новый столбец resultant_category.

In [11]:
def get_resultant_category(row):
    # Список источников для категорий
    sources = [
        row['BitDefender'], row['Dr.Web'], row['Sophos'], row['Forcepoint ThreatSeeker'],
        row['Comodo Valkyrie Verdict'], row['Xcitium Verdict Cloud'], row['Websense ThreatSeeker'],
        row['alphaMountain.ai'], row['Webroot'], row['sophos']
    ]
    # Удаляем значения NaN и категории 'uncategorized'
    filtered_sources = [cat for cat in sources if pd.notna(cat) and cat != 'uncategorized']
    
    if filtered_sources:
        # Определяем наиболее часто встречающуюся категорию
        most_common_category, _ = Counter(filtered_sources).most_common(1)[0]
        return most_common_category
    else:
        # Если подходящих категорий нет, возвращаем значение из столбца 'cats gpt'
        return row['cats gpt']

# Применяем функцию для создания результирующего столбца
globals()['df']['resultant_category'] = globals()['df'].apply(get_resultant_category, axis=1)


5. Матрица совпадений категорий

Этот код вычисляет матрицу совпадений категорий для каждого URL, сравнивая значения в различных столбцах категорий. Сначала определяется список категорий, затем для каждой строки подсчитывается количество совпадений между категориями с помощью функции count_matches. Результаты накапливаются в матрице match_matrix. Итоговая матрица преобразуется в DataFrame для удобства анализа и сохраняется в Excel-файл 'category_match_matrix.xlsx'.

In [13]:
# Список всех категорий
category_columns = df.columns[1:]  # Предполагается, что первый столбец - URL

# Функция для подсчета совпадений между категориями для каждого URL
def count_matches(row):
    matches = np.zeros((len(category_columns), len(category_columns)))
    for i, col1 in enumerate(category_columns):
        for j, col2 in enumerate(category_columns):
            if i != j and pd.notna(row[col1]) and row[col1] == row[col2]:
                matches[i, j] = 1
    return matches

# Инициализируем матрицу совпадений
match_matrix = np.zeros((len(category_columns), len(category_columns)))

# Заполняем матрицу совпадений
df[category_columns].apply(lambda row: count_matches(row), axis=1).apply(lambda x: match_matrix.__iadd__(x))

# Преобразуем матрицу совпадений в DataFrame для удобства
match_df = pd.DataFrame(match_matrix, index=category_columns, columns=category_columns)

# Сохраняем результат в Excel
match_df.to_excel('category_match_matrix.xlsx')

print("Матрица совпадений категорий сохранена в 'category_match_matrix.xlsx'")


Матрица совпадений категорий сохранена в 'category_match_matrix.xlsx'


In [14]:
match_df

,cats base,cats gpt,BitDefender,Dr.Web,Sophos,Forcepoint ThreatSeeker,Comodo Valkyrie Verdict,Xcitium Verdict Cloud,Websense ThreatSeeker,alphaMountain.ai,Webroot,sophos,alpha_1,alpha_2,resultant_category
cats base,0.0,1337.0,10.0,3.0,7.0,10.0,0.0,8.0,0.0,0.0,0.0,0.0,12.0,5.0,434.0
cats gpt,1337.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,16.0,1.0,1095.0
BitDefender,10.0,0.0,0.0,1.0,29.0,25.0,0.0,19.0,1.0,0.0,0.0,0.0,43.0,13.0,1442.0
Dr.Web,3.0,1.0,1.0,0.0,67.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,75.0
Sophos,7.0,2.0,29.0,67.0,0.0,494.0,0.0,15.0,0.0,0.0,0.0,0.0,344.0,239.0,730.0
Forcepoint ThreatSeeker,10.0,0.0,25.0,2.0,494.0,0.0,0.0,84.0,0.0,0.0,0.0,8.0,567.0,248.0,1543.0
Comodo Valkyrie Verdict,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0
Xcitium Verdict Cloud,8.0,0.0,19.0,3.0,15.0,84.0,0.0,0.0,0.0,0.0,0.0,0.0,76.0,15.0,167.0
Websense ThreatSeeker,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,9.0,61.0
alphaMountain.ai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


6. Регулярные выражения для анализа сайтов

In [15]:
df = df.rename(columns = {'url': 'req_host'})

In [16]:
df_results = df[['req_host','resultant_category']]

Этот код создает регулярное выражение на основе столбца req_host для каждого URL в DataFrame. Если req_host представляет собой IP-адрес, возвращается сам адрес. Для других хостов регулярное выражение строится с использованием домена и топ-уровня домена (TLD). Затем добавляются новые столбцы в DataFrame: url_regex_mod с созданными регулярными выражениями и req_host_combined, который копирует значения из req_host. Также переименовывается столбец resultant_category в cat_popular. В завершение преобразованный DataFrame сохраняется в CSV-файл 'combined_hosts_with_fixed_regex_v40.csv'.

In [17]:
# Функция для создания регулярного выражения на основе `req_host`
def create_url_regex(host):
    if re.match(r'^\d+\.\d+\.\d+\.\d+$', host):
        return host
    host_parts = host.split('.')
    tld = host_parts[-1]
    domain = host_parts[-2]
    regex = rf"^([\w-]+\.)*(www\.)?{domain}\.({tld})$"
    return regex


# Преобразование данных
df_results['req_host_combined'] = df_results['req_host']
df_results['url_regex_mod'] = df_results['req_host'].apply(create_url_regex)
df_results['url'] = df_results['req_host']
df_results = df_results.rename(columns={'resultant_category': 'cat_popular'})
# Вывод преобразованной таблицы
df_results[['req_host_combined', 'url_regex_mod', 'url', 'cat_popular']].to_csv(
    'combined_hosts_with_fixed_regex_v40.csv', index=False
)

C:\Users\rozhk\AppData\Local\Temp\ipykernel_4648\878153277.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_results['req_host_combined'] = df_results['req_host']
C:\Users\rozhk\AppData\Local\Temp\ipykernel_4648\878153277.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_results['url_regex_mod'] = df_results['req_host'].apply(create_url_regex)
C:\Users\rozhk\AppData\Local\Temp\ipykernel_4648\878153277.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

Итоговый результат
Файл combined_hosts_with_fixed_regex_v40.csv содержит объединенные данные с итоговыми категориями, что позволяет использовать его для дальнейшего анализа и автоматизации процессов классификации.